# Notebook E-tivity 1 CE4021

Student name: Peter O'Mahony

Student ID: 8361967

<hr style="border:2px solid gray"> </hr>

## Imports

In [1]:
#None

<hr style="border:2px solid gray"> </hr>

## Task

# Option 1: Symbolic calculation of the derivative
## Write 2 Python functions:

*The first one to symbolically calculate the derivative of a polynomial with one variable. The input should be a polynomial and the output should also be a polynomial.*

*The second one to evaluate (i.e. get the numerical value) of a polynomial for a given value of its variable. The input should be a polynomial and a value (point at which to evaluate the polynomial). The output should be a scalar.*

*Test your code with a few salient polynomials (minimum of 3) for which you have calculated the derivative of these polynomials manually.*

### Initial thoughts:
Split it into subtasks
1. handle input of equation
2. differentiate the input
3. display the resulting prettified input and output
4. handle input of variable value
5. calculate the result
6. display resulting scalar
7. display a set of test results

Data structures I am considering are lists and tuples.
I want to maximise the flexibility of the solution so I will ask the user to specify the powers rather than determine them impicitly from the order of the coefficients.

I'd like to use tuples because the data are static after entered and tuples are more efficient than lists and I don't need to do much manipulation of the structures.

I'll try to use UTF8 characters to prettify the results (thereby avoiding any libraries). Note Jason's idea to sort the results by descending power as is conventional.

## Final Assumptions
We expect the coefficients and powers to be positive or negative floating point numbers.

# Get coding

In [2]:
def pick_a_sample_equation() -> list:
    """
    Picks a scientifically random entry from a vast list of creative and surprising polynomials. 
    Sometimes. Well... *** whispers *** It picks the same one every time 
    """
    # TODO: hmm looks tricky to find a random number with just native python functions so think harder and 
    # just return this one for now
    return [(8,4),(-3,3),(6,2),(2,1),(11,0)]

In [3]:
print(72*'-') # output a line for clearer user input
print('This program requests that you enter a representation of a polynomial \n'
      'and then it will differentiate it and display the result. It includes \n'
      'some built-in tests to validate its functionality.\n\n'
      'Enter a pair of real numbers a,b such that \n'
      'a is the coefficient and \n'
      'b is the power')

equation = [] # store the tuples entered by the user

# get a pair until the user indicates that they are finished by pressing return twice
while True:
    term = input("Enter in the format: a,b (or leave blank to finish) >")
    if term:
        # split the input string at the comma and convert to a tuple of floats
        # (sounds like the kind of thing overheard late in pub)
        term_as_tuple = tuple( float(part) for part in term.split(',')) 
        # TODO: perhaps add some error checking here e.g. only 2 numeric parts
        equation.append(term_as_tuple)
    else:
        break

if len(equation) == 0:
        # user entered nothing so let's show what we prepared earlier
        equation = pick_a_sample_equation()
        print("You may be a little tired. Allow me to help.  We'll pretend you entered this and say nothing.")

# show the user what we understood from their entries in a format similar to that which they used
print('You entered:', equation) 

------------------------------------------------------------------------
This program requests that you enter a representation of a polynomial 
and then it will differentiate it and display the result. It includes 
some built-in tests to validate its functionality.

Enter a pair of real numbers a,b such that 
a is the coefficient and 
b is the power
Enter in the format: a,b (or leave blank to finish) >
You may be a little tired. Allow me to help.  We'll pretend you entered this and say nothing.
You entered: [(8, 4), (-3, 3), (6, 2), (2, 1), (11, 0)]


We will be showing the user some equations but, as libraries are prohibited, we roll our own prettifier to reassure the user that we are interpreting their entries correctly.

In [4]:
def pom_superscript(power: int) -> str:
    # creating translation table using https://symbl.cc/en/unicode/blocks/latin-1-supplement/
    # this function is free to all in exchange for a life hack🙂
    # e.g. wrap iceberg lettuce in tinfoil and store in your fridge to add weeks to its lifespan!
    """
    Returns a UTF8 string of superscript characters for display in a Jupyter Notebook based on the 
    power value supplied as a parameter.
    """
    translate_table = str.maketrans("0123456789-.", 
                                    "\u2070\u00B9\u00B2\u00B3\u2074\u2075\u2076\u2077\u2078\u2079\u207B\u00B7")
    return str(power).translate(translate_table)

Let's do some testing

In [20]:
print(f'y = x{pom_superscript(4)} + x{pom_superscript(-8.25)}')

y = x⁴ + x⁻⁸·²⁵


OK enough distractions, get on with it now.
We'll use the above function in our main prettifier that displays the polynomial

In [21]:
# show the input in a prettier way
def pretty_equation(equation: list, sort: bool=False, rhs_var: str='x', lhs_var: str='y') -> None:
    """
    This returns a user accessible string which tidies our equation and omits zero value tuples
    sort sorts the result in order of descending power of terms
    var1 changes the default (x) variable name and
    var2 changes the default (y) variable name on the left hand side of the equation
    """
    if sort:
        # identify the key we want to use for sorting
        sort_by_power = lambda the_power : the_power[1]
        # sort in order of descending power
        equation.sort(key=sort_by_power, reverse=True)
    
    result = f' {lhs_var} ='
    for (idx,(coeff,power)) in enumerate(equation):
        if power == None or coeff == 0: # omit tuple if it has a zero value
            continue
            
        if coeff < 0:
            sign =  ' - '
        else:
            if idx > 0:
                sign =  ' + '
            else:
                sign =  ' '
            
        if coeff == 1 and power != 0:
            coeff = ''; # a coefficient of 1 is conventionally omitted if it is a multiplier
        else:
            coeff = abs(coeff) # we don't need a negative sign displayed because we are handling it explicitly
            if int(coeff) == coeff: # convert to int to drop decimals unless they are relevant
                coeff = int(coeff)

        if power == 0:
            result += f'{sign}{coeff}'
        elif power == 1:
            result += f'{sign}{coeff}{rhs_var}'
        else:
            result += f'{sign}{coeff}{rhs_var}{pom_superscript(power)}'
    return result

Here comes the real work of the program

In [22]:
# differentiate a tuple by multiplying power by coeff and decrementing power
def differentiate_tuple(coeff: int, power: int) -> tuple:
    """
    Differentiates a tuple by multiplying power by coeff and decrementing power and returns a new tuple
    """
    return (coeff*power, power-1) # all this work for this little tiny bit of code? My goodness me

Now we apply our functions to the data we gathered earlier and show off

In [14]:
def differentiate_poly(equation: list) -> list:
    differentiated_result = []
    for (coeff, power) in equation:
        new_coeff, new_power = differentiate_tuple(coeff, power)
        if new_coeff != 0:
            differentiated_result.append((new_coeff, new_power))
    return differentiated_result


In [24]:
print("You entered\n",pretty_equation(equation));

differentiated_result = differentiate_poly(equation)
    
print("and when differentiated this becomes\n", pretty_equation(differentiated_result, sort=True))

print("or as a list of tuples:")
print(differentiated_result)

You entered
  y = 8x⁴ - 3x³ + 6x² + 2x + 11
and when differentiated this becomes
  y = 32x³ - 9x² + 12x + 2
or as a list of tuples:
[(32, 3), (-9, 2), (12, 1), (2, 0)]


Now we will run our own tests. Don't make a big deal of it if our tests are more sophisticated than the polynomial the user entered. Just act normal.

In [30]:
test_set = (
            [(6,3),(-9,1),(4,0)],
            [(3,3),(2,2),(1,1),(0,0)],
            [(4,7),(-3,-7),(9,1)],
            [(1,-4),(-9,-3),(8,-2),(12,0)],
            [],
            [(5,1),(3,-3),(-5,1)],
            [(3,0.25)],
            [(17,0),(7,2),(2,1000),(9,61)],
            [(7,-4),(3,4.5),(4,-3.25),(1.5,2)]
            )
    
print(72*'-','\nSAMPLE TESTS OF DIFFERENTIATION\n',72*'-') # TODO: why does the third line have a leading space?

for test_id, test_poly in enumerate(test_set):
    print(f"\tTest   {test_id}: {pretty_equation(test_poly)}" )
    print(f"\tResult {test_id}: {pretty_equation(differentiate_poly(test_poly), sort=True)}\n")


------------------------------------------------------------------------ 
SAMPLES TESTS OF DIFFERENTIATION
 ------------------------------------------------------------------------
	Test   0:  y = 6x³ - 9x + 4
	Result 0:  y = 18x² - 9

	Test   1:  y = 3x³ + 2x² + x
	Result 1:  y = 9x² + 4x + 1

	Test   2:  y = 4x⁷ - 3x⁻⁷ + 9x
	Result 2:  y = 28x⁶ + 9 + 21x⁻⁸

	Test   3:  y = x⁻⁴ - 9x⁻³ + 8x⁻² + 12
	Result 3:  y = - 16x⁻³ + 27x⁻⁴ - 4x⁻⁵

	Test   4:  y =
	Result 4:  y =

	Test   5:  y = 5x + 3x⁻³ - 5x
	Result 5:  y = 5 - 5 - 9x⁻⁴

	Test   6:  y = 3x⁰·²⁵
	Result 6:  y = 0.75x⁻⁰·⁷⁵

	Test   7:  y = 17 + 7x² + 2x¹⁰⁰⁰ + 9x⁶¹
	Result 7:  y = 2000x⁹⁹⁹ + 549x⁶⁰ + 14x

	Test   8:  y = 7x⁻⁴ + 3x⁴·⁵ + 4x⁻³·²⁵ + 1.5x²
	Result 8:  y = 13.5x³·⁵ + 3x - 13x⁻⁴·²⁵ - 28x⁻⁵



Now, part 2 (I can hardly wait) we'll use our equations and apply some values to see the magic happen.

In [16]:
def evaluate_polynomial(equation: list, sample_x: int) -> int:
    """
    Interprets the list of tuples in the equation and applies the value in sample_x
    """
    result = 0
    for (coeff, power) in equation:
        if sample_x == 0 or coeff == 0: 
            continue
        result += coeff * (sample_x**power) if power>= 0 else coeff * ((1/sample_x)**abs(power))
    return result

In [41]:
test_set = (-2,0,1,3,10,0.5)

print(72*'-','\nSAMPLE TESTS PART 2 (to 2 nearest decimal places)\n',72*'-') # TODO: why does the third line have a leading space?

for test_id, test_value in enumerate(test_set):
    print(f"Test {test_id} using value {test_value}")
    print(f"\t polynomial result:     {evaluate_polynomial(equation, test_value):12.2f}")
    print(f"\t differentiated result: {evaluate_polynomial(differentiated_result, test_value):12.2f}")

------------------------------------------------------------------------ 
SAMPLE TESTS PART 2 (to 2 nearest decimal places)
 ------------------------------------------------------------------------
Test 0 using value -2
	 polynomial result:           183.00
	 differentiated result:      -314.00
Test 1 using value 0
	 polynomial result:             0.00
	 differentiated result:         0.00
Test 2 using value 1
	 polynomial result:            24.00
	 differentiated result:        37.00
Test 3 using value 3
	 polynomial result:           638.00
	 differentiated result:       821.00
Test 4 using value 10
	 polynomial result:         77631.00
	 differentiated result:     31222.00
Test 5 using value 0.5
	 polynomial result:            13.62
	 differentiated result:         9.75


<hr style="border:2px solid gray"> </hr>

## Reflection

*Write your reflection in below cell. With reference to the Rubric for E-tivity 1:*
- *Provide an accurate description of your code with advantages and disadvantages of design choices.*
- *Compare your approach to alternative (peer) approaches.*
- *Clearly describe how you have used your peers' work/input and how this has affected your own understanding / insights.*

*If you have not used peer input, you may state this, but your submission history in Gitlab should clearly show this is the case.*

## TBD